In [25]:
import os
import sys
if os.path.abspath('tusp-pddl-generator') not in sys.path:
    sys.path.append(os.path.abspath('tusp-pddl-generator'))
import json
import time
import subprocess

from src.parser import *

from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import networkx as nx
import numpy as np

from relativeImp import relativeImp

from dominance_analysis import Dominance_Datasets
from dominance_analysis import Dominance

class ENHSPException(Exception):
    pass

In [26]:
def create_yard(ntrains, ntracks, ncm, sg, pg, ptracks, stracks):
    gen = YardGenerator(
        ntrains,
        (80,),
        ntracks,
        (80,),
        int(ntracks*ncm),
        sg,
        sg+pg,
        stracks, 
        ptracks
    )

    yard = ShuntingYard(
        'test_yard',
        'dom_n',
        DomainConfig.Numeric,
        DirectionStrategy.Aside,
        True,
        TrackOccupationStrategy.ORDER,
        0
    )
    yard.load_generator(gen)
    yard.generate_instance()
    yard.draw_graph()
    yard.visualize(savefig=True)

    service_tracks = [t for t in yard.tracks if t.servicing]
    
    short_path_len =  []
    num_short_path = []
    all_path_len = []
    num_all_path = []
    for t in service_tracks:
        all_short_path = list(nx.all_shortest_paths(yard.graph, 'track_entry', t.name))
        all_sim_path = list(nx.all_simple_paths(yard.graph, 'track_entry', t.name))
        num_short_path.append(len(all_short_path))
        short_path_len.append(len(all_short_path[0]))
        num_all_path.append(len(all_sim_path))
        all_path_len.append(np.mean([len(x) for x in all_sim_path]))
    return yard, np.mean(short_path_len), np.mean(num_short_path), np.mean(all_path_len), np.mean(num_all_path)


def collect_enhsp_statistics():
    start_time = time.time()

    result = subprocess.run(["java", "-jar", "./ENHSP-Public/enhsp-dist/enhsp.jar",
                                            "-sp", "output.plan", "-h", "hadd", "-s",
                                            "wa_star_4", "-gro", "internal", "-ha", "true",
                                            "-o", "dom_n.pddl", "-f", "test_yard.pddl"], 
                                            timeout=30,
                                            stdout=subprocess.PIPE
                                            )
                            
    dt = time.time() - start_time  
    try:
        os.remove("output.plan")
    except:
        raise ENHSPException(f'{result.returncode}\n{result.stdout}\n{result.stderr}')

    output_lines = result.stdout.decode('utf8').split('\n')
    num_actions = 0
    for line in output_lines:
        if line.startswith('|A|'):
            num_actions = int(line.split(':')[1])

    return dt, num_actions


In [27]:
NUM_TRAINS = [2, 3]
NUM_TRACKS = [15]#, 20, 25]
NUM_CONNECTIONS_MULTIPLIER = [1.5, 2.5]
SERVICE_GOALS = [3]
PARKING_GOALS = [-1]
NUM_PARKING_TRACKS_MULTIPLIER = [0.65, 0.8]
NUM_SERVICE_TRACKS = [1]
REPITITIONS = 3

os.chdir("ENHSP-Public")
subprocess.call("./compile")
os.chdir("..")


results = {
    'ntrains':[],
    'ntracks':[],
    'nactions':[],
    'npath_short':[],
    'avgpath_short':[],
    'npath_simple':[],
    'avgpath_simple':[],
    'ncm':[],
    'steps':[],
    'config':[],
    'ptracks':[],
    'stracks':[],
    'runtime':[],
}

fail_count = 0
ACTION_BINWIDTH = 50

with tqdm(total=len(NUM_TRAINS)*len(NUM_TRACKS)*len(NUM_CONNECTIONS_MULTIPLIER)*len(PARKING_GOALS)*len(SERVICE_GOALS)*len(NUM_PARKING_TRACKS_MULTIPLIER)*len(NUM_SERVICE_TRACKS)*REPITITIONS) as pbar:
    for ntrains in NUM_TRAINS:
        for ntracks in NUM_TRACKS:
            for ncm in NUM_CONNECTIONS_MULTIPLIER:
                for sgoal in SERVICE_GOALS:
                    for pgoal in PARKING_GOALS:
                        for ptracks_m in NUM_PARKING_TRACKS_MULTIPLIER:
                            for stracks in NUM_SERVICE_TRACKS:
                                for _ in range(REPITITIONS):
                                    ptracks = int(ntracks * ptracks_m)
                                    fails = 0
                                    while fails < 2:
                                        try:
                                            yard, spl, nsp, apl, nap = create_yard(ntrains, ntracks, ncm, sgoal, pgoal, ptracks, stracks)
                                            rt, nactions = collect_enhsp_statistics()
                                            break
                                        except Exception as e:
                                            fails +=1 
                                    pbar.update(1)
                                    if fails >= 2:
                                        continue
                                    results['ntrains'].append(ntrains)
                                    results['ntracks'].append(ntracks)
                                    results['nactions'].append(nactions)
                                    results['npath_short'].append(nsp)
                                    results['avgpath_short'].append(spl)
                                    results['npath_simple'].append(nap)
                                    results['avgpath_simple'].append(apl)
                                    results['ncm'].append(ncm)
                                    results['steps'].append(abs(sgoal) + abs(pgoal))
                                    results['config'].append(1+int(pgoal>0))
                                    results['ptracks'].append(ptracks_m)
                                    results['stracks'].append(stracks)
                                    results['runtime'].append(rt)

print("Fail counter: ", fail_count)

df = pd.DataFrame(results)

for col in ['nactions', 'npath_short', 'avgpath_short', 'npath_simple', 'avgpath_simple']:
    counts, edges = np.histogram(df[col], bins=10)
    df[col] = [np.searchsorted(edges, v, side="left") for v in df[col]]


df.to_csv('results.csv')

Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.
mkdir: cannot create directory ‘enhsp-dist’: File exists
100%|██████████| 24/24 [02:12<00:00,  5.52s/it]

Fail counter:  0


In [31]:
df.groupby(['ntrains', 'nactions', 'npath_simple'])['runtime'].mean()

ntrains  nactions  npath_simple
2        0         0               0.464140
                   1               0.481009
         5         3               0.792773
                   4               0.647117
                   5               0.726720
3        4         1               1.247808
         10        2               1.589648
                   4               1.905836
                   5               1.151909
                   6               1.904719
                   10              2.120719
Name: runtime, dtype: float64